In [1]:
from Historic_Crypto import HistoricalData as HD
import pandas as pd
import requests
import json
from datetime import datetime
import matplotlib.pyplot as plt
import scipy

`Etherscan API Information and Data`
- The Etherscan API collects data from the blockchain. 
- In this case, I am using the action 'tokentx' which collects all fo the transaction data for a specifc token conducted by a single crypto wallet address.
- The link to the Etherscan API documentation is as follows: https://docs.etherscan.io/api-endpoints/accounts 
- The function 'etherScanAPICall()' calls the API endpoint and returns a JSON fof the reponse which contains the transaction data. The JSON is also stored in a JSON file. 
- To put simply, as of right now, we only want to aggregate the number of transactions that occurred daily (regardless of buy or sell).

In [2]:
def etherScanAPICall(wallet_address):
    API_KEY = 'JATAZ8XWIBGBTY15YJJEABCDPIF8SCK7S3'
    MATIC_CONTRACT_ADDRESS = '0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0'
    # WALLET_ADDRESS = '0x9507c04b10486547584c37bcbd931b2a4fee9a41'
    url = 'https://api.etherscan.io/api?'

    parameters = {
        'module':'account',
        'action':'tokentx',
        'contractaddress':MATIC_CONTRACT_ADDRESS,
        'address':wallet_address,
        'page':'1',
        'offset':'10000',
        'startblock':'0',
        'endblock':'27025780',
        'sort':'asc',
        'apikey':API_KEY
    }

    responseJSON = requests.request("GET", url, params=parameters).json()

    print("# of Transactions:",len(responseJSON.get("result")))

    with open("transactionsResponse.json", "w") as write_file:
        json.dump(responseJSON, write_file, indent=4, sort_keys = True)

    return responseJSON


In [3]:
def convert_UNIX_to_DateTime(unix):
    return datetime.utcfromtimestamp(int(unix)).strftime('%Y-%m-%d %H:%M:%S') # returns type str

In [4]:
def getDailyTransactionVolume(transactionData):
    dailyTransactionVolume = {}

    for transaction in transactionData.get('result'):
        timeStamp = convert_UNIX_to_DateTime(transaction.get("timeStamp"))
        timeStamp = timeStamp[0:10]
        if (timeStamp in dailyTransactionVolume):
            cnt = dailyTransactionVolume.get(timeStamp) + 1
            dailyTransactionVolume[timeStamp] = cnt
        else:
            dailyTransactionVolume[timeStamp] = 1

    return dailyTransactionVolume

In [5]:
rawTransactionData = etherScanAPICall('0x9507c04b10486547584c37bcbd931b2a4fee9a41')
# rawTransactionData = etherScanAPICall('0x5666ed92c83af9dab61601c87bf7769ad57103f9')

rawDailyTransactionVolume = getDailyTransactionVolume(rawTransactionData)

print(len(rawDailyTransactionVolume))
print(rawDailyTransactionVolume)

# of Transactions: 2745
66
{'2022-08-24': 2, '2022-09-23': 69, '2022-09-24': 48, '2022-09-25': 40, '2022-09-26': 96, '2022-09-27': 34, '2022-09-28': 3, '2022-09-29': 19, '2022-09-30': 103, '2022-10-01': 9, '2022-10-02': 26, '2022-10-03': 60, '2022-10-04': 69, '2022-10-05': 4, '2022-10-07': 3, '2022-10-08': 27, '2022-10-10': 1, '2022-10-12': 2, '2022-10-18': 1, '2022-10-21': 19, '2022-10-22': 24, '2022-10-23': 29, '2022-10-24': 28, '2022-10-25': 63, '2022-10-26': 49, '2022-10-27': 51, '2022-10-28': 46, '2022-10-29': 56, '2022-10-30': 71, '2022-10-31': 44, '2022-11-01': 29, '2022-11-02': 87, '2022-11-03': 116, '2022-11-04': 162, '2022-11-05': 110, '2022-11-06': 73, '2022-11-07': 88, '2022-11-08': 97, '2022-11-11': 37, '2022-11-12': 5, '2022-11-14': 48, '2022-11-15': 13, '2022-11-16': 20, '2022-11-17': 30, '2022-11-18': 2, '2022-11-19': 8, '2022-11-20': 23, '2022-11-21': 1, '2022-11-23': 21, '2022-11-24': 61, '2022-11-25': 34, '2022-11-26': 14, '2022-11-27': 15, '2022-11-28': 7, '2022-11-

In [6]:
def getStartAndEndDate(transactionData):
    startDate = next(iter(transactionData)) 
    endDate = list(transactionData)[-1]
    return [startDate, endDate]

`Historic_Crypto Python Library Information and Data`
- The Historic_Crypto Python Libray uses the Coinbase Pro API to retrieve the historical data of many different types of cryptocurrencies.
- The function 'getCryptoHistoricalData()' calls the library function 'retrieve_data()' to return a dataframe of the historical pricing data of the token entered.
- The function also safe the dataframe data to a CSV file.

In [7]:
def getCryptoHistoricalData(token, startDate, endDate):
    data = HD(token,86400,startDate+"-00-00",endDate+"-00-00").retrieve_data()
    data.to_csv('tokenPriceData.csv')
    return data

In [8]:
startAndEndDate = getStartAndEndDate(rawDailyTransactionVolume)

rawTokenData = getCryptoHistoricalData('MATIC-USD', startAndEndDate[0], startAndEndDate[1])

Checking input parameters are in the correct format.
Formatting Dates.
Checking if user supplied is available on the CoinBase Pro API.
Connected to the CoinBase Pro API.
Ticker 'MATIC-USD' found at the CoinBase Pro API, continuing to extraction.
Retrieved Data from Coinbase Pro API.
Returning data.


In [9]:
print(rawTokenData)

               low    high    open   close      volume
time                                                  
2022-08-24  0.8019  0.8406  0.8320  0.8135  25232711.6
2022-08-25  0.8052  0.8321  0.8135  0.8190  22782683.0
2022-08-26  0.7562  0.8484  0.8193  0.7667  58825306.5
2022-08-27  0.7570  0.8147  0.7666  0.8060  34756485.9
2022-08-28  0.7777  0.8337  0.8062  0.7827  30160499.1
...            ...     ...     ...     ...         ...
2022-12-06  0.8965  0.9214  0.9117  0.9176  25489220.1
2022-12-07  0.8707  0.9198  0.9177  0.8978  37556021.2
2022-12-08  0.8874  0.9301  0.8979  0.9278  29494131.3
2022-12-09  0.9080  0.9346  0.9278  0.9097  21652345.5
2022-12-10  0.9016  0.9101  0.9095  0.9064   5324477.7

[109 rows x 5 columns]
